# Introduction to dimensionality reduction

## Setup
### Imports

In [ ]:
import pandas as pd                                     # for dataset manipulation (DataFrames)
import numpy as np                                      # allows some mathematical operations
import matplotlib.pyplot as plt                         # library used to display graphs
import seaborn as sns                                   # more convenient visualisation library for dataframes
import missingno as msno                                # utility library for missing values
from sklearn.model_selection import train_test_split    # for classification
from sklearn.neighbors import KNeighborsClassifier      # for classification

### Loading the dataset

In [ ]:
fives = np.loadtxt("fives.txt", delimiter=",")
sixes = np.loadtxt("sixes.txt", delimiter=",")

In [ ]:
index = 0
image = np.reshape(fives[index, :],(16, 16))

plt.imshow(image, cmap="gray")
plt.show()